In [8]:
###########################
### MODULES NECESSAIRES ###
###########################


# Import the necessary libraries
import numpy as np
import pandas as pd
import math
import fonctionsSupervisedLearning2 as fsl2

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn import svm

from auxFonctions import AminoAcid

In [9]:
# Read data from a file into a list of entries
with open('data/SIG_13.red', 'r') as file:
    entries = file.read().split('\n   ')

In [10]:
##############################
## RECUPERATION DES DONNÉES ##
##############################

# Process each entry
processed_entries = [fsl2.process_entry(entry) for entry in entries]

# Create a DataFrame
df = pd.DataFrame(processed_entries)

# Get the position of the cleavage site
cleavage_site_position = df['Annotation'].apply(lambda x: x.find('C'))

# Split the primary structure into a list of amino acids
amino_acid_seq = df['Primary Structure'].apply(lambda x: list(x))

## Traitement des données

Put words in vector and vice versa

In [11]:
df_exploitable = fsl2.convert_df_to_vectors2(df)

In [12]:
n = 15
p,q = 13, 2  # p+q = 15

In [14]:
# Create a DataFrame to store, for each primary structure, the neihborhood of the cleavage site
# The neighborhood is defined as the word of length p+q starting p letters before the cleavage site
correct_neighborhood = pd.Series()
for i, seq in amino_acid_seq.items():
    correct_neighborhood[i] = ''.join(seq[cleavage_site_position[i]-p:cleavage_site_position[i]+q])

C:\Users\rabas\AppData\Local\Temp\ipykernel_15104\1829963092.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  correct_neighborhood = pd.Series()


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# Create a DataFrame to store, for each primary structure, a sequence that is not the neighborhood of the cleavage site
incorrect_neighborhood = pd.Series()
decalage  = [1,2,3,4,5, -1,-2,-3,-4, -5]
for i, seq in amino_acid_seq.items():
    dec = np.random.choice(decalage)
    dec = 0 if cleavage_site_position[i]-13 - dec < 0 else dec
    incorrect_neighborhood[i] = ''.join(seq[cleavage_site_position[i]-p-dec:cleavage_site_position[i]+q-dec])

In [ ]:
#print(correct_neighborhood.get(4))
print(fsl2.word_to_vector(incorrect_neighborhood.get(52)))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

## Train the models

In [ ]:
X_train, X_test, bool_train, bool_test = fsl2.test_train_split_random_pos2(df_exploitable, 15, random_state=42)

count_true = 0
count_all = 0
for t in bool_train:
    count_all += 1
    if t:
        count_true += 1

print(count_true)
print(count_all)

602
1126


In [ ]:
#svm_model_nei, accuracy_nei = fsl2.create_model2(15, df_exploitable, random_state=42, nb_letters = 26, kernel_neighbor= 'rbf',C_nei = 1)

df_exploitable_100 = df_exploitable.iloc[:,1:100]

C_list = [0.1,1,2]
kernel_list = ['linear', 'rbf', 'sigmoid', 'poly']
n_list = [15, 17, 19]
results = []

for C_nei in C_list:
    for kernel_nei in kernel_list:
        for n in n_list:
            svm_model_nei, accuracy_nei = fsl2.create_model2(n, df_exploitable.iloc[:,1:100], kernel_nei, C_nei)
            results.append((C_nei, kernel_nei, n, accuracy_nei))

results_df = pd.DataFrame(results, columns=['C_nei','kernel_nei', 'n', 'accuracy_nei'])
results_df.to_csv('results2.csv', index=True)


In [ ]:
best_results = results_df.loc[(results_df['accuracy_nei']).idxmax()]
print(best_results)

C_nei                2.0
kernel_nei       sigmoid
n                     17
accuracy_nei    0.907801
Name: 31, dtype: object


In [ ]:
svm_model_nei, accuracy_nei = fsl2.create_model2(15, df_exploitable,'sigmoid', 2.0)

In [ ]:
count = 0
for i in range(0, len(correct_neighborhood)):
    X = fsl2.word_to_vector(df["Primary Structure"].get(i))
    if fsl2.find_cleavage2(X,svm_model_nei,15):
        count += 1

print(count)


/Users/mathiasperez/Documents/GitHub/442-2-Protein-cleavage/fonctionsSupervisedLearning2.py:132: SyntaxWarning: invalid escape sequence '\ '
  '''


ValueError: X has 390 features, but SVC is expecting 494 features as input.

In [ ]:
count_fn = 0
for i in range(0, len(correct_neighborhood)):
    test = fsl2.is_neighborhood2(correct_neighborhood.get(i), svm_model_nei, 15)
    if not(test):
        count_fn += 1

print("Nombre de faux négatifs :" + str(count_fn))

count_fp = 0
for i in range(0, len(incorrect_neighborhood)):
    try : 
        test = fsl2.is_neighborhood2(incorrect_neighborhood.get(i), svm_model_nei, 15)
        if test:
            count_fp += 1
    except:
        print("Erreur" + str(i))
        #print(incorrect_neighborhood.get(i))


print("Nombre de faux positifs :" + str(count_fp))
print("Nombre total :" + len(correct_neighborhood))




Nombre de faux négatifs :191
Nombre de faux positifs :449


TypeError: can only concatenate str (not "int") to str

In [ ]:
# print(fsl.find_cleavage(test, threshold = 0.1))
pos_pred = np.array([fsl.find_cleavage(X,svm_model_in, svm_model_pos, threshold = 0.0, n= 17) for X in df_exploitable['P_Structure_vector'][:100]])

in_pred = 1-np.isnan(pos_pred)

pos_pred[in_pred == 0] = 0

print(accuracy_score(df_exploitable["Annotation_pos"][:100], pos_pred))

0.53
